In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler

#모델링 모듈
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, average_precision_score, confusion_matrix
import shap
from imblearn.over_sampling import SMOTE

#1. data_load
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
data = pd.read_csv('Churn_Modelling.csv',index_col=0)

#2. data_preprocessing
# missing D(4개 행 제거)
data.isnull().sum()
nan_data = data.dropna()
print('결측 처리 :' ,nan_data.shape)

# duplicated D(2행 제거)
nan_data[nan_data.duplicated()]
new_data = nan_data[~nan_data.duplicated()]
print('중복 처리 :',new_data.shape)

#encoding : gender-label ( Female = 0, male = 1) /  geography-onehot
le = LabelEncoder()
new_data['Gender'] = le.fit_transform(new_data["Gender"])

oe = OneHotEncoder()
oe.fit(new_data[['Geography']])
geo_csr = oe.transform(new_data[['Geography']])
csr_df = pd.DataFrame(geo_csr.toarray(), columns = oe.get_feature_names_out())
df = new_data.reset_index(drop=True)  # df 인덱스 초기화
csr_df = csr_df.reset_index(drop=True)  # csr_df 인덱스 초기화
inco_df = pd.concat([df,csr_df],axis=1)

#check
int_data = inco_df.drop(columns=['CustomerId', 'Surname','Geography'])
X = int_data.drop("Exited", axis=1)
y_true = int_data[['Exited']]
print('전처리 완료:',X.shape,y_true.shape)
print('----------------------------')

#4.data engineering
#tester split
X_train, X_test, y_train, y_test = train_test_split(X,y_true,test_size = 0.3, random_state= 42)
print('데이터 분리 후 크기 : ',X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#minmaxscaling - 적용유무 검토 
target_features = ["CreditScore", "Age", "Balance", "EstimatedSalary"]
X_train_sc = X_train.copy()
X_test_sc = X_test.copy()

mn_sc = MinMaxScaler()
X_train_sc[target_features] = mn_sc.fit_transform(X_train[target_features])
X_test_sc[target_features] = mn_sc.transform(X_test[target_features]) #테스터는 정규화 학습 안함 

#Over Sampling
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_sc, y_train)
print("SMOTE 적용 후 데이터 크기:", X_train_res.shape, y_train_res.shape)
print(f'original : {y_train.value_counts()}')
print(f'smote : {y_train_res.value_counts()}')

print('----------------------------')
#모델 성능 비교 (실행 : 9.8초)
results = {} #결과 저장 

models = {
    "LogisticRegression": LogisticRegression(),
    "kNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGB": XGBClassifier(random_state=42),
    "GBM": GradientBoostingClassifier(random_state=42)
}

for model_name, model in models.items():
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test_sc)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    results[model_name] = {'Accuracy': acc, 'f1-score': f1, 'recall' : recall, 'precision': precision}

#데이터 프레임으로 출력 
results_df = pd.DataFrame(results).transpose()
pd.set_option('display.max_colwidth', None) #문자열 안잘리게 출력 

print('[분류 모델별 성능 비교]')
results_df.round(5)

결측 처리 : (9998, 13)
중복 처리 : (9996, 13)
전처리 완료: (9996, 12) (9996, 1)
----------------------------
데이터 분리 후 크기 :  (6997, 12) (2999, 12) (6997, 1) (2999, 1)
SMOTE 적용 후 데이터 크기: (11140, 12) (11140, 1)
original : Exited
0         5570
1         1427
Name: count, dtype: int64
smote : Exited
0         5570
1         5570
Name: count, dtype: int64
----------------------------
[분류 모델별 성능 비교]
                    Accuracy  f1-score   recall  precision
LogisticRegression   0.69957   0.47707  0.67377    0.36927
kNN                  0.71691   0.46503  0.60492    0.37769
SVM                  0.75358   0.54966  0.73934    0.43744
DecisionTree         0.76459   0.51776  0.62131    0.44379
RandomForest         0.83795   0.62090  0.65246    0.59226
XGB                  0.83828   0.60408  0.60656    0.60163
GBM                  0.81294   0.60960  0.71803    0.52963
